In [1]:
%pylab
from matplotlib import animation

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [6]:
def diff_P(P, sun_orbit):
    G = 1.0
    M_1 = .5
    M_2 = .3
    x_1, y_1, x_2, y_2 = sun_orbit
    x, y, vx, vy = P
    r3_1 = (sqrt((x-x_1)**2 + (y-y_1)**2)**3)        
    r3_2 = (sqrt((x-x_2)**2 + (y-y_2)**2)**3)
    ax_1 = -G*M_1*(x-x_1)/r3_1
    ax_2 = -G*M_2*(x-x_2)/r3_2
    ay_1 = -G*M_1*(y-y_1)/r3_1
    ay_2 = -G*M_2*(y-y_2)/r3_2
    ax =ax_1 + ax_2
    ay = ay_1 + ay_2
    return array([vx, vy, ax, ay])

In [3]:
def calculate_orbit(steps, h, x=3., y=0., vx=0, vy=.5):
    r= .5 # could pass this in through the parameters of calculate_orbit
    sun_orbit = sun_rot(steps, h, r)
    orbit = empty((steps, 4))
    P = array([x, y, vx, vy])
    for i in xrange(steps):
        orbit[i] = P
        Ptilde = P + h*diff_P(P, sun_orbit[i])
        P += h*(diff_P(P, sun_orbit[i]) + diff_P(Ptilde, sun_orbit[i]))/2.
    return orbit

In [4]:
def sun_rot(steps,h, r):
    sun_orbit = empty((steps,4))
   
    for t in xrange(steps):
        x_1 = r*cos(t*h)
        x_2 = -r*cos(t*h)
        y_1 = r*sin(t*h)
        y_2 = -r*sin(t*h)
        sun_orbit[t] = x_1, y_1, x_2, y_2
    
    return sun_orbit

In [5]:
steps = 5000
h = .1
fig, ax = subplots(figsize=(6, 6))
ax.set_aspect('equal')
ax.set_axis_bgcolor('k')
# "point" repesents the planet 
point, = plot([], [], 'ro')
line, = plot([], [], 'cyan')
sun, = plot([], [], 'yo', ms=8)
sun_2, = plot([],[], "yo", ms = 8)
# The size of the displayed area is fixed at the start, to avoid resizing during the animation
size = 4
xlim(-size, size)
ylim(-size, size)
# Calculate the complete orbit in advance.
orbit = calculate_orbit(steps, h)
sun_orbit = sun_rot(steps, h, .5)


# Initialization function for the start of the animation
def init():
    point.set_data([], [])
    line.set_data([], [])
    sun.set_data([], [])
    sun_2.set_data([], [])
    return line, point, sun, sun_2,

# Function to update the data for each frame of the animation
def animate(i):
    point.set_data(orbit[i, 0], orbit[i, 1])
    line.set_data(orbit[:i, 0], orbit[:i, 1])
    sun.set_data(sun_orbit[i, 0], sun_orbit[i, 1])
    sun_2.set_data(sun_orbit[i,2], sun_orbit[i, 3])
    return line, point, sun, sun_2,

# This is called to generate and run the animation
animation.FuncAnimation(fig, animate, init_func=init, frames=steps, interval=20, blit=True, repeat=True)